In [1]:
import numpy as np
import pandas as pd
import math
import random
import torch
import torch.nn as nn
import torch.optim as optim
import time

# Change dataset file Here

In [2]:
# Load the dataset
file_path = 'vopd.csv'
df = pd.read_csv(file_path)

In [3]:
df

,source,target,weight
0,0,1,70
1,1,2,362
2,2,3,362
3,3,4,362
4,3,15,49
5,4,5,357
6,4,15,27
7,5,6,353
8,5,8,16
9,5,11,16


In [4]:
# Define CoreGraph class
class CoreGraph:
    def __init__(self, cores, edges, bandwidths):
        self.cores = cores
        self.edges = edges
        self.bandwidths = bandwidths

    def get_bandwidth(self, core1, core2):
        try:
            index = self.edges.index((core1, core2))
            return self.bandwidths[index]
        except ValueError:
            return 0

# Define helper function to extract core graph information from the dataset
def initialize_core_graph(df):
    source_column = 'source'
    target_column = 'target'
    bandwidth_column = 'weight'

    cores = list(set(df[source_column]).union(set(df[target_column])))
    edges = list(zip(df[source_column], df[target_column]))
    bandwidths = list(df[bandwidth_column])

    core_graph = CoreGraph(cores, edges, bandwidths)
    return core_graph

# Initialize CoreGraph
core_graph = initialize_core_graph(df)

# Display the core graph details
print("Cores:", core_graph.cores)
print("Edges:", core_graph.edges)
print("Bandwidths:", core_graph.bandwidths)

Cores: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Edges: [(0, 1), (1, 2), (2, 3), (3, 4), (3, 15), (4, 5), (4, 15), (5, 6), (5, 8), (5, 11), (6, 7), (7, 8), (7, 9), (8, 9), (10, 11), (10, 14), (11, 12), (12, 13), (12, 14), (13, 14)]
Bandwidths: [70, 362, 362, 362, 49, 357, 27, 353, 16, 16, 300, 313, 500, 313, 16, 16, 16, 157, 16, 16]


In [5]:
# Define helper functions to calculate coordinates and hops
def get_coordinates(router_id, X, Y):
    """Convert router ID to (x, y) coordinates."""
    return (router_id // X, router_id % X)

def get_hops(router1, router2):
    """Calculate the Manhattan distance (hops) between two routers."""
    x1, y1 = router1
    x2, y2 = router2
    return abs(x1 - x2) + abs(y1 - y2)

In [6]:

def calculate_grid_dimensions(num_cores):
    """Calculate the grid dimensions (X, Y) for the given number of cores."""
    X = int(math.ceil(math.sqrt(num_cores)))
    Y = int(math.ceil(num_cores / X))
    return X, Y

# Calculate grid dimensions based on the highest core number in the dataset
def calculate_grid_size(df):
    source_column = 'source'
    target_column = 'target'
    
    highest_number = max(df[source_column].max(), df[target_column].max())
    num_cores = highest_number + 1  # Core numbers are 0-based, so add 1
    
    X, Y = calculate_grid_dimensions(num_cores)
    
    return highest_number, num_cores, X, Y

# Calculate the grid dimensions
highest_number, num_cores, X, Y = calculate_grid_size(df)

# Print grid dimensions
print(f"Highest Core Number: {highest_number}")
print(f"Number of Cores: {num_cores}")
print(f"Grid Dimensions: {X} x {Y}")

Highest Core Number: 15
Number of Cores: 16
Grid Dimensions: 4 x 4


In [7]:
def get_coordinates(router_id, X, Y):
    """Convert router ID to (x, y) coordinates."""
    return (router_id // X, router_id % X)

def verify_router_coordinates(mapping, X, Y):
    for router, core in enumerate(mapping):
        router_coords = get_coordinates(router, X, Y)
        print(f"Core: {core}, Router ID: {router}, Router Coordinates: {router_coords}")

# Test with a sample mapping array
sample_mapping_array = np.array([5, 6, 3, 4, 7, 8, 0, 1, 2])

# Verify router coordinates
verify_router_coordinates(sample_mapping_array, X, Y)


Core: 5, Router ID: 0, Router Coordinates: (0, 0)
Core: 6, Router ID: 1, Router Coordinates: (0, 1)
Core: 3, Router ID: 2, Router Coordinates: (0, 2)
Core: 4, Router ID: 3, Router Coordinates: (0, 3)
Core: 7, Router ID: 4, Router Coordinates: (1, 0)
Core: 8, Router ID: 5, Router Coordinates: (1, 1)
Core: 0, Router ID: 6, Router Coordinates: (1, 2)
Core: 1, Router ID: 7, Router Coordinates: (1, 3)
Core: 2, Router ID: 8, Router Coordinates: (2, 0)


In [8]:
def compute_communication_cost(mapping, core_graph, X, Y):
    """Compute the total communication cost for a given mapping."""
    cost = 0
    for (core1, core2) in core_graph.edges:
        # Get the router indices from the mapping array
        router_id1 = np.where(mapping == core1)[0][0]
        router_id2 = np.where(mapping == core2)[0][0]
        # Get the router coordinates
        router1 = get_coordinates(router_id1, X, Y)
        router2 = get_coordinates(router_id2, X, Y)
        # Calculate hops between the routers
        hops = get_hops(router1, router2)
        # Get the bandwidth for the edge
        bandwidth = core_graph.get_bandwidth(core1, core2)
        # Calculate the cost as hops * bandwidth
        cost += hops * bandwidth
    return cost

def evaluate_population(population, core_graph, X, Y):
    fitness_scores = []
    for mapping in population:
        cost = compute_communication_cost(mapping, core_graph, X, Y)
        fitness_scores.append(cost)
    return fitness_scores

# # Example usage
# population_ex = [np.array([0, 1, 2, 3, 4, 5, 6, 7, 8]), np.array([5, 6, 3, 4, 7, 8, 0, 1, 2])]
# fitness_scores_ex = evaluate_population(population_ex, core_graph, X, Y)

# # Print fitness scores
# for i, score in enumerate(fitness_scores_ex):
#     print(f"Individual {i}: Communication Cost = {score}")

In [9]:
class ActorNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(ActorNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_size)  # Output actions

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        actions = torch.tanh(self.fc3(x))  # Actions in range [-1, 1]
        return actions

class CriticNetwork(nn.Module):
    def __init__(self, state_size):
        super(CriticNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Output a single value representing the estimated communication cost

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        value = self.fc3(x)
        return value

# Example state and action sizes
state_size = X*Y  # Number of features representing the state
action_size =  X*Y  # Number of actions to propose (one for each element to mutate)
actor = ActorNetwork(state_size, action_size)
critic = CriticNetwork(state_size)



In [10]:
print(actor)

ActorNetwork(
  (fc1): Linear(in_features=16, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
)


In [11]:
def individual_to_state(individual):
    return torch.tensor(individual, dtype=torch.float32)

# # Test with a sample individual
# sample_individual = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
# sample_state = individual_to_state(sample_individual)
# print(sample_state)


In [12]:
def propose_actions_with_actor(individual, actor):
    state = torch.tensor(individual, dtype=torch.float32)  # Convert individual to a tensor
    actions = actor(state)
    return actions.detach().numpy()  # Convert actions to NumPy array

# # Test with a sample individual
# sample_actions = propose_actions_with_actor(sample_individual)
# print("Sample actions:", sample_actions)


In [13]:
# Tournament selection function
def tournament_selection(population, fitness_scores, tournament_size):
    tournament = np.random.choice(len(population), tournament_size, replace=False)
    tournament_fitness = [fitness_scores[i] for i in tournament]
    winner_index = tournament[np.argmin(tournament_fitness)]  # Assuming lower fitness is better
    return population[winner_index]

In [14]:
def crossover(parent1, parent2, actions1, actions2):
    size = len(parent1)
    crossover_point = int(size * (actions1[0] + 1) / 2)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

    # Resolve duplicates in child1
    unique_child1, indices1 = np.unique(child1, return_index=True)
    missing1 = np.setdiff1d(np.arange(size), unique_child1)
    for i in range(len(child1)):
        if i not in indices1:
            child1[i] = missing1[0]
            missing1 = missing1[1:]

    # Resolve duplicates in child2
    unique_child2, indices2 = np.unique(child2, return_index=True)
    missing2 = np.setdiff1d(np.arange(size), unique_child2)
    for i in range(len(child2)):
        if i not in indices2:
            child2[i] = missing2[0]
            missing2 = missing2[1:]

    return child1, child2

# # Test crossover with duplicates resolution
# sample_individual2 = np.array([8, 7, 6, 5, 4, 3, 2, 1, 0])
# sample_actions2 = propose_actions_with_actor(sample_individual2)
# child1, child2 = crossover(sample_individual, sample_individual2, sample_actions, sample_actions2)
# print("Child 1:", child1)
# print("Child 2:", child2)



# Mutation rate here

In [15]:
def mutate(mapping, actions, mutation_rate=0.5):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(len(mapping), size=2, replace=False)
        mapping[idx1], mapping[idx2] = mapping[idx2], mapping[idx1]
    return mapping

# # Test mutation
# mutated_individual = mutate(sample_individual, sample_actions)
# print("Mutated individual:", mutated_individual)


In [16]:
def initialize_population(pop_size, num_routers):
    population = []
    for _ in range(pop_size):
        mapping = np.random.permutation(num_routers)
        population.append(mapping)
    return population

# Define the population size and number of routers
pop_size = 10
num_routers = X * Y

# Initialize the population
population = initialize_population(pop_size, num_routers)

# Print the initialized population
for i, individual in enumerate(population):
    print(f"Individual {i}: {individual}")


Individual 0: [ 9  1  4  6  7 15 13 14  0 12 10  2 11  3  8  5]
Individual 1: [10  5  1 11 14  2 15 13  6  4  7  9  8 12  3  0]
Individual 2: [ 1  0 11  2  3  4  9  7 14  5 12 13 15 10  8  6]
Individual 3: [11  0 12  4  6  1 10  3  7 15 13  9  8 14  5  2]
Individual 4: [14 15  8 13 10  3  4  5  0 11  2 12  1  9  6  7]
Individual 5: [ 6  3 10 14 13  7  4  0  2  8 12  9 11  1 15  5]
Individual 6: [15 12 10  0  5  7  1 13  3  9  4  2  6 14 11  8]
Individual 7: [12  0  1 13  3  5  4 14 15  2  6 11 10  7  9  8]
Individual 8: [ 1  9 15  3 10 12  8  6  7 11  5  4 14 13  0  2]
Individual 9: [ 5  9 14  6 11  7 13  4  3  8  1 15  2 12 10  0]


In [17]:
def generate_new_population(population, fitness_scores, actor, tournament_size):
    new_population = []
    for _ in range(len(population) // 2):
        parent1 = tournament_selection(population, fitness_scores, tournament_size)
        parent2 = tournament_selection(population, fitness_scores, tournament_size)
        actions1 = propose_actions_with_actor(parent1, actor)
        actions2 = propose_actions_with_actor(parent2, actor)

        # Perform crossover with duplicates resolution
        child1, child2 = crossover(parent1, parent2, actions1, actions2)

        # Perform mutation on the children
        child1 = mutate(child1, actions1)
        child2 = mutate(child2, actions2)

        new_population.append(child1)
        new_population.append(child2)

    return new_population

# # Example initial population
# initial_population = [
#     np.array([0, 1, 2, 4, 5, 8, 7, 6, 3]),
#     np.array([0, 1, 2, 4, 5, 3, 8, 7, 6]),
#     np.array([8, 7, 6, 5, 4, 3, 2, 1, 0]),
#     np.array([1, 2, 3, 4, 5, 6, 7, 8, 0])
# 

# Define the population size and number of routers
pop_size = 10
num_routers = X * Y
sample_population=initialize_population(pop_size, num_routers)
sample_fitness_scores=evaluate_population(sample_population, core_graph, X, Y)
tournament_size=3

# Initialize the Actor network
state_size = X*Y
action_size = X*Y
actor = ActorNetwork(state_size, action_size)
# # Generate new population
# new_population = generate_new_population(initial_population)
# print("New population:", new_population)


# Test the generate_new_population function
new_population = generate_new_population(sample_population, sample_fitness_scores, actor, tournament_size)
print("New population:")
for i, individual in enumerate(new_population):
    print(f"Individual {i}: {individual}")

New population:
Individual 0: [ 2  7  9  6 13  8 10  1 15 11  4 12  3  0  5 14]
Individual 1: [ 0 11  3  1  6 12  4 10 14  7  9  5 13  2  8 15]
Individual 2: [13 15  7  3 10 12 11  9 14  1  8  5  4  2  0  6]
Individual 3: [ 0 11  3  1  6 12  4 14  7  8  2 10 13  5  9 15]
Individual 4: [ 1  2  5  4  8 15 11  0  6  9 13 10 14  3  7 12]
Individual 5: [15  7  3  6 11  4 12  0  9 10  1  2  5  8 13 14]
Individual 6: [ 1  2  5  4  8 15 11 14  6  9 13 10  0  3  7 12]
Individual 7: [11  9 10  4 12 14  0 15  7  5  8  3  1  2  6 13]
Individual 8: [ 1  2  8  4  5 15 11 14  6  9 13 10  0  3  7 12]
Individual 9: [ 0 11  9  1  6 12  4 10 14  7  8  5  2  3 13 15]


In [18]:
def evaluate_population(population, core_graph, X, Y):
    fitness_scores = []
    for mapping in population:
        cost = compute_communication_cost(mapping, core_graph, X, Y)
        fitness_scores.append(cost)
    return fitness_scores

# Example usage
fitness_scores = evaluate_population(new_population, core_graph, X, Y)
for i, score in enumerate(fitness_scores):
    print(f"Individual {i}: Communication Cost = {score}")


Individual 0: Communication Cost = 8727
Individual 1: Communication Cost = 9015
Individual 2: Communication Cost = 10413
Individual 3: Communication Cost = 8271
Individual 4: Communication Cost = 9882
Individual 5: Communication Cost = 9856
Individual 6: Communication Cost = 9732
Individual 7: Communication Cost = 9636
Individual 8: Communication Cost = 9760
Individual 9: Communication Cost = 9933


In [19]:
# new_population

In [20]:


def run_genetic_algorithm_with_actor_critic(pop_size, grid_size, core_graph, X, Y, generations, tournament_size):
    """Run the genetic algorithm with Actor-Critic model."""
    # Define the Actor and Critic Networks
    class ActorNetwork(nn.Module):
        def __init__(self, state_size, action_size):
            super(ActorNetwork, self).__init__()
            self.fc1 = nn.Linear(state_size, 128)
            self.fc2 = nn.Linear(128, 64)
            self.fc3 = nn.Linear(64, action_size)  # Output actions

        def forward(self, state):
            x = torch.relu(self.fc1(state))
            x = torch.relu(self.fc2(x))
            actions = torch.tanh(self.fc3(x))  # Actions in range [-1, 1]
            return actions

    class CriticNetwork(nn.Module):
        def __init__(self, state_size):
            super(CriticNetwork, self).__init__()
            self.fc1 = nn.Linear(state_size, 128)
            self.fc2 = nn.Linear(128, 64)
            self.fc3 = nn.Linear(64, 1)  # Output a single value representing the estimated communication cost

        def forward(self, state):
            x = torch.relu(self.fc1(state))
            x = torch.relu(self.fc2(x))
            value = self.fc3(x)
            return value

    # Initialize the Actor and Critic networks
    state_size = grid_size
    action_size = grid_size
    actor = ActorNetwork(state_size, action_size)
    critic = CriticNetwork(state_size)

    # Define loss functions and optimizers for the Actor and Critic
    critic_loss_fn = nn.MSELoss()
    actor_loss_fn = nn.MSELoss()
    critic_optimizer = optim.Adam(critic.parameters(), lr=0.001)
    actor_optimizer = optim.Adam(actor.parameters(), lr=0.001)

    # Initialize population
    population = initialize_population(pop_size, grid_size)

    # Initialize tracking of the best individual and its cost
    best_individual = None
    best_cost = float('inf')
    best_costs_per_epoch = []
    runtimes_per_epoch = []

    # Training loop for the Critic and Actor
    for epoch in range(generations):
        # Record start time
        start_time = time.time()

        fitness_scores = evaluate_population(population, core_graph, X, Y)
        new_population = generate_new_population(population, fitness_scores, actor, tournament_size)

        for i, individual in enumerate(new_population):
            state = individual_to_state(individual)
            actions = propose_actions_with_actor(individual, actor)

            # Apply mutation and crossover
            proposed_individual = mutate(individual, actions)
            proposed_state = individual_to_state(proposed_individual)

            # Compute actual fitness
            actual_fitness = compute_communication_cost(proposed_individual, core_graph, X, Y)
            target = torch.tensor([actual_fitness], dtype=torch.float32)

            # Train Critic
            critic_optimizer.zero_grad()
            predicted_value = critic(proposed_state)
            critic_loss = critic_loss_fn(predicted_value, target)
            critic_loss.backward()
            critic_optimizer.step()

            # Train Actor (using the feedback from Critic)
            actor_optimizer.zero_grad()
            actor_loss = actor_loss_fn(critic(proposed_state), target.detach())
            actor_loss.backward()
            actor_optimizer.step()

            # Track the best individual
            if actual_fitness < best_cost:
                best_cost = actual_fitness
                best_individual = proposed_individual

        # Update the population for the next generation
        population = new_population

        # Record the best cost of the epoch
        best_costs_per_epoch.append(best_cost)

        # Record end time and calculate duration
        end_time = time.time()
        epoch_runtime = end_time - start_time
        runtimes_per_epoch.append(epoch_runtime)

#         print(f"Runtime This Generation: {epoch_runtime} seconds")

        # Print best result of the epoch
        print(f"Epoch {epoch}: Best Communication Cost = {best_cost}")

    print(f"Best individual overall: {best_individual}")
    print(f"Best communication cost overall: {best_cost}")

    return best_individual, best_cost, best_costs_per_epoch, runtimes_per_epoch


# For run multiple trial

In [21]:

# Variables to store results
best_costs = []
total_runtimes = []
all_best_costs_per_gen = []
convergence_points = []

# Run the genetic algorithm 10 times and record results
for i in range(10):
    print(f"Run {i + 1}/10")
    
    best_mapping, best_cost, best_costs_per_epoch, runtimes_per_epoch = run_genetic_algorithm_with_actor_critic(
    pop_size=100,
    grid_size=X * Y,
    core_graph=core_graph,
    X=X,
    Y=Y,
    generations=100,
    tournament_size=4)

    

    total_runtime = sum(runtimes_per_epoch)
    
    best_costs.append(best_cost)
    total_runtimes.append(total_runtime)
    # Find convergence point
    min_index = best_costs_per_epoch.index(min(best_costs_per_epoch))
    convergence_points.append(min_index)
    all_best_costs_per_gen.append(best_costs_per_epoch.copy())

Run 1/10
Epoch 0: Best Communication Cost = 6645
Epoch 1: Best Communication Cost = 5403
Epoch 2: Best Communication Cost = 5403
Epoch 3: Best Communication Cost = 5277
Epoch 4: Best Communication Cost = 5277
Epoch 5: Best Communication Cost = 5277
Epoch 6: Best Communication Cost = 5131
Epoch 7: Best Communication Cost = 4554
Epoch 8: Best Communication Cost = 4554
Epoch 9: Best Communication Cost = 4554
Epoch 10: Best Communication Cost = 4554
Epoch 11: Best Communication Cost = 4554
Epoch 12: Best Communication Cost = 4554
Epoch 13: Best Communication Cost = 4554
Epoch 14: Best Communication Cost = 4554
Epoch 15: Best Communication Cost = 4554
Epoch 16: Best Communication Cost = 4554
Epoch 17: Best Communication Cost = 4401
Epoch 18: Best Communication Cost = 4273
Epoch 19: Best Communication Cost = 4273
Epoch 20: Best Communication Cost = 4273
Epoch 21: Best Communication Cost = 4273
Epoch 22: Best Communication Cost = 4273
Epoch 23: Best Communication Cost = 4245
Epoch 24: Best Co

Epoch 98: Best Communication Cost = 4041
Epoch 99: Best Communication Cost = 4041
Best individual overall: [ 1  0 12 13  2 11 10 14  3  4  8  9 15  5  6  7]
Best communication cost overall: 4041
Run 3/10
Epoch 0: Best Communication Cost = 6287
Epoch 1: Best Communication Cost = 6250
Epoch 2: Best Communication Cost = 5281
Epoch 3: Best Communication Cost = 5281
Epoch 4: Best Communication Cost = 5281
Epoch 5: Best Communication Cost = 5281
Epoch 6: Best Communication Cost = 5186
Epoch 7: Best Communication Cost = 5186
Epoch 8: Best Communication Cost = 5186
Epoch 9: Best Communication Cost = 5186
Epoch 10: Best Communication Cost = 5186
Epoch 11: Best Communication Cost = 5186
Epoch 12: Best Communication Cost = 4932
Epoch 13: Best Communication Cost = 4780
Epoch 14: Best Communication Cost = 4780
Epoch 15: Best Communication Cost = 4780
Epoch 16: Best Communication Cost = 4780
Epoch 17: Best Communication Cost = 4780
Epoch 18: Best Communication Cost = 4748
Epoch 19: Best Communicatio

Epoch 93: Best Communication Cost = 4143
Epoch 94: Best Communication Cost = 4111
Epoch 95: Best Communication Cost = 4111
Epoch 96: Best Communication Cost = 4111
Epoch 97: Best Communication Cost = 4111
Epoch 98: Best Communication Cost = 4111
Epoch 99: Best Communication Cost = 4111
Best individual overall: [ 0  1  9  8 12  2  7  6 13  3  4  5 14 10 15 11]
Best communication cost overall: 4111
Run 5/10
Epoch 0: Best Communication Cost = 5853
Epoch 1: Best Communication Cost = 5853
Epoch 2: Best Communication Cost = 5853
Epoch 3: Best Communication Cost = 5652
Epoch 4: Best Communication Cost = 4591
Epoch 5: Best Communication Cost = 4591
Epoch 6: Best Communication Cost = 4591
Epoch 7: Best Communication Cost = 4591
Epoch 8: Best Communication Cost = 4591
Epoch 9: Best Communication Cost = 4274
Epoch 10: Best Communication Cost = 4274
Epoch 11: Best Communication Cost = 4274
Epoch 12: Best Communication Cost = 4219
Epoch 13: Best Communication Cost = 4219
Epoch 14: Best Communicatio

Epoch 88: Best Communication Cost = 4041
Epoch 89: Best Communication Cost = 4041
Epoch 90: Best Communication Cost = 4041
Epoch 91: Best Communication Cost = 4041
Epoch 92: Best Communication Cost = 4041
Epoch 93: Best Communication Cost = 4041
Epoch 94: Best Communication Cost = 4041
Epoch 95: Best Communication Cost = 4041
Epoch 96: Best Communication Cost = 4041
Epoch 97: Best Communication Cost = 4041
Epoch 98: Best Communication Cost = 4041
Epoch 99: Best Communication Cost = 4041
Best individual overall: [ 1  2  3 15  0  5  4 11  8  6 12 10  9  7 13 14]
Best communication cost overall: 4041
Run 7/10
Epoch 0: Best Communication Cost = 6184
Epoch 1: Best Communication Cost = 6036
Epoch 2: Best Communication Cost = 6036
Epoch 3: Best Communication Cost = 5562
Epoch 4: Best Communication Cost = 5390
Epoch 5: Best Communication Cost = 5107
Epoch 6: Best Communication Cost = 5107
Epoch 7: Best Communication Cost = 5107
Epoch 8: Best Communication Cost = 5107
Epoch 9: Best Communicatio

Epoch 83: Best Communication Cost = 4159
Epoch 84: Best Communication Cost = 4159
Epoch 85: Best Communication Cost = 4137
Epoch 86: Best Communication Cost = 4137
Epoch 87: Best Communication Cost = 4105
Epoch 88: Best Communication Cost = 4105
Epoch 89: Best Communication Cost = 4105
Epoch 90: Best Communication Cost = 4105
Epoch 91: Best Communication Cost = 4105
Epoch 92: Best Communication Cost = 4105
Epoch 93: Best Communication Cost = 4105
Epoch 94: Best Communication Cost = 4105
Epoch 95: Best Communication Cost = 4089
Epoch 96: Best Communication Cost = 4089
Epoch 97: Best Communication Cost = 4089
Epoch 98: Best Communication Cost = 4041
Epoch 99: Best Communication Cost = 4041
Best individual overall: [15  3  2  1  5  4 11  0  6  8 12 13  7  9 10 14]
Best communication cost overall: 4041
Run 9/10
Epoch 0: Best Communication Cost = 6296
Epoch 1: Best Communication Cost = 6207
Epoch 2: Best Communication Cost = 6052
Epoch 3: Best Communication Cost = 5482
Epoch 4: Best Communi

Epoch 78: Best Communication Cost = 4159
Epoch 79: Best Communication Cost = 4159
Epoch 80: Best Communication Cost = 4159
Epoch 81: Best Communication Cost = 4159
Epoch 82: Best Communication Cost = 4159
Epoch 83: Best Communication Cost = 4159
Epoch 84: Best Communication Cost = 4159
Epoch 85: Best Communication Cost = 4159
Epoch 86: Best Communication Cost = 4127
Epoch 87: Best Communication Cost = 4127
Epoch 88: Best Communication Cost = 4127
Epoch 89: Best Communication Cost = 4127
Epoch 90: Best Communication Cost = 4127
Epoch 91: Best Communication Cost = 4127
Epoch 92: Best Communication Cost = 4127
Epoch 93: Best Communication Cost = 4127
Epoch 94: Best Communication Cost = 4127
Epoch 95: Best Communication Cost = 4127
Epoch 96: Best Communication Cost = 4127
Epoch 97: Best Communication Cost = 4127
Epoch 98: Best Communication Cost = 4127
Epoch 99: Best Communication Cost = 4127
Best individual overall: [ 8  6  5  4  9  7 15  3 10 11  1  2 14 13 12  0]
Best communication cost

In [28]:
# Create a DataFrame
results_df = pd.DataFrame({
    'Best Cost': best_costs,
    'Total Runtime': total_runtimes,
    'Convergence Point': convergence_points
})

In [29]:
results_df

,Best Cost,Total Runtime,Convergence Point
0,4089,43.373308,36
1,4041,49.578198,77
2,4063,49.315274,55
3,4111,49.395735,94
4,4105,51.987435,38
5,4041,52.631311,49
6,4171,60.089239,39
7,4041,53.528161,98
8,4057,62.478948,84
9,4127,54.034100,86


In [30]:
# Convert the list of lists to a DataFrame
df_all_cost = pd.DataFrame(all_best_costs_per_gen).transpose()
# Add headers
df_all_cost.columns = [f'Run {i+1}' for i in range(df_all_cost.shape[1])]

In [31]:
df_all_cost[0:5]

,Run 1,Run 2,Run 3,Run 4,Run 5,Run 6,Run 7,Run 8,Run 9,Run 10
0,6645,5584,6287,6615,5853,5691,6184,6089,6296,5716
1,5403,5584,6250,6257,5853,5691,6036,6049,6207,5716
2,5403,5259,5281,5657,5853,5483,6036,5986,6052,5716
3,5277,5259,5281,5657,5652,5349,5562,5585,5482,5716
4,5277,5092,5281,5467,4591,5026,5390,5585,5403,5716


In [32]:
df_all_cost.shape

(100, 10)

# Export to excel

In [33]:
results_df.to_excel('Summary_GA+RL_name.xlsx', index=False)

In [34]:
df_all_cost.to_excel('rawdata_GA+RL_name.xlsx', index=False)

# For single run

In [22]:
# # Example usage
# best_mapping, best_cost, best_costs_per_epoch, runtimes_per_epoch = run_genetic_algorithm_with_actor_critic(
#     pop_size=200,
#     grid_size=X * Y,
#     core_graph=core_graph,
#     X=X,
#     Y=Y,
#     generations=100,
#     tournament_size=4
# )

# print("Best Mapping:", best_mapping)
# print("Best Cost:", best_cost)
# # print("Best Costs Per Epoch:", best_costs_per_epoch)


In [23]:
best_mapping

array([ 8,  6,  5,  4,  9,  7, 15,  3, 10, 11,  1,  2, 14, 13, 12,  0])

In [24]:
compute_communication_cost(best_mapping, core_graph, X, Y)

4127

In [25]:
best_costs_per_epoch[0:5]

[5716, 5716, 5716, 5716, 5716]

In [26]:
 sum(runtimes_per_epoch)

54.034099817276

In [27]:
 sum(runtimes_per_epoch)/len(runtimes_per_epoch)

0.54034099817276